In [1]:
!wget https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv

--2019-01-04 00:48:20--  https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3093160 (2.9M) [text/plain]
Saving to: ‘colloquial-indonesian-lexicon.csv.1’

colloquial-indonesi 100%[===================>]   2.95M  --.-KB/s    in 0.1s    

2019-01-04 00:48:21 (29.2 MB/s) - ‘colloquial-indonesian-lexicon.csv.1’ saved [3093160/3093160]



In [2]:
!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git
!bash f7b7c7758a46da49f84bc68b47997d69/pytorch041_cuda92_colab.sh

fatal: destination path 'f7b7c7758a46da49f84bc68b47997d69' already exists and is not an empty directory.
--2019-01-04 00:48:27--  https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64.deb?-yARSlRPH2hgt2UnnOmBmw3q0NNp2x86SQ0O5tdy_gDjDfwaCSQPp7Oaw0v76W3hfAJL4qcSRABEe9c1hutpaniQGD1F4u2ZWBAGwNtKbUiFcPftYJg8sTjVKbNGR_7kh8FTQrf0m-kw846fPDZbzcqGg1UYNlQHB6w3pYj3QsBv7N3odpAOr4X5kZfM6768ojhxJBMwlARGCtVB_rftXQ [following]
--2019-01-04 00:48:29--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64.deb?-yAR

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
import torch.nn.functional as F
import torch
from torch import nn, optim

In [0]:
device = torch.device(
    'cuda' if torch.cuda.is_available() else 'cpu'
)

In [0]:
df = pd.read_csv('colloquial-indonesian-lexicon.csv')

In [8]:
df.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [9]:
df.groupby('category1').agg('count')

,slang,formal,In-dictionary,context,category2,category3
category1,,,,,,
abreviasi,7162,7162,7162,7162,7162,7162
afiksasi,1089,1089,1089,1089,1089,1089
akronim,256,256,256,256,256,256
anaptiksis,404,404,404,404,404,404
coinage,50,50,50,50,50,50
elongasi,1906,1906,1906,1906,1906,1906
homofon,169,169,169,169,169,169
metatesis,13,13,13,13,13,13
modifikasi vokal,626,626,626,626,626,626


In [0]:
X = df['slang']
y = df['formal']
SOS_TOKEN = 0
PAD_TOKEN = ord('*')

In [11]:
def apply_padding(_str, n_max_char, zero_char='*'):
  len_str = len(_str)
  if len_str >= n_max_char:
    return _str[:n_max_char]
  
  else:
    return _str + ''.join([zero_char for t in range(n_max_char - len_str)])
  
print(apply_padding('abcd', 10))

abcd******


In [12]:
max_len_slang = X.apply(len).max()
max_len_formal = y.apply(len).max()
print('max_len_slang: {}, max_len_formal {}'.format(max_len_slang, max_len_formal))

max_len_slang: 59, max_len_formal 22


In [0]:
def lord(x):
  """make string into integer"""
  return [SOS_TOKEN] + [ord(x1) for x1 in x]

In [0]:
X = X.apply(apply_padding, args=(max_len_slang,)).apply(list).apply(np.array).apply(lord)
y = y.apply(apply_padding, args=(max_len_slang,)).apply(list).apply(np.array).apply(lord)

In [0]:
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.3)

In [0]:
def to_torch(x, device=device):
  return torch.from_numpy(np.array(x.values.tolist())).to(device)

In [0]:
class Encoder(nn.Module):
  def __init__(self, hidden_size, vocab_size, embed_dim):
    super(Encoder, self).__init__()
    self.embedding = nn.Embedding(vocab_size, vocab_size)
    self.gru = nn.GRU(vocab_size * embed_dim, hidden_size)
    
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim
    
  def forward(self, x, hidden):
    embed = self.embedding(x).view(1, 1, -1)
    output, hidden = self.gru(embed, hidden)
    return output, hidden
  
  def init_hidden(self, device):
    return torch.zeros(1,1,self.hidden_size, device=device)
  
class Decoder(nn.Module):
  def __init__(self, hidden_size, vocab_size, embed_dim, out_size):
    super(Decoder, self).__init__()
    self.embedding = nn.Embedding(vocab_size, vocab_size)
    self.gru = nn.GRU(vocab_size * embed_dim, hidden_size)
    self.l1 = nn.Linear(hidden_size, out_size)
    self.sm = nn.LogSoftmax(dim=1)
    self.hidden_size = hidden_size
    
  def forward(self, x, hidden):
    out = self.embedding(x).view(1,1,-1)
    out = F.relu(out)
    out, out_hidden = self.gru(out, hidden)
    out = self.sm(self.l1(out[0]))
    
    return out, out_hidden
  
  def init_hidden(self, device):
    return torch.zeros(1,1,self.hidden_size, device=device)
  
class SimpleSeq2Seq(nn.Module):
  def __init__(self, encoder, decoder, vocab_size):
    super(SimpleSeq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.vocab_size = vocab_size
    
  def forward(self, x, y, device=device):
    max_len_translation = y.shape[0] 
    encoder_hidden = self.encoder.init_hidden(device)
    encoded, encoder_hidden = self.encoder(x, encoder_hidden)
    
    # buffer for full outputs
    outputs = list()
    # make tensor to have dimension
    # or in other words make into list (in tensor)
    _input = y[:1]
    # skip the first token since it will be SOS
    for t in range(1, max_len_translation):
      output, decoder_hidden = self.decoder(x, encoder_hidden)
      outputs.append(output.view(-1))
      _input = output
      
    return torch.cat(outputs).view(max_len_translation - 1, -1).to(device)

In [0]:
# hidden_size = 
input_size = 60
vocab_size = 122
embed_dim = 60
hidden_dim = 256

encoder = Encoder(hidden_dim, vocab_size, embed_dim).to(device)
decoder = Decoder(hidden_dim, vocab_size, embed_dim, vocab_size).to(device)
seq = SimpleSeq2Seq(encoder, decoder, vocab_size).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

optimizer = optim.Adam(seq.parameters())

epochs = 10

In [0]:
Xt_torch, yt_torch = to_torch(X_train), to_torch(y_train)

In [57]:
epoch_loss = 0
max_len_translation = input_size
seq.train()

for e in range(epochs):
  len_iter = 0
  for src, trg in zip(Xt_torch, yt_torch):
    len_iter += 1
    src = src.to(device)
    trg = trg.to(device)
    
    optimizer.zero_grad()
    
    output = seq(src, trg)
    
    loss = criterion(output, trg[1:])
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
  
  print('loss: {}'.format(epoch_loss/len(len_iter)))

RuntimeError: ignored

In [0]:
encoder_hidden = encoder.init_hidden(device)
decoder_hidden = decoder.init_hidden(device)
running_loss = 0
train_losses = []
test_losses = []
epochs = 20
loss = 0

for e in range(epochs):
  for slang, target in zip(to_torch(X_train), to_torch(y_train)):
    slang = slang.to(device)
    target = target.to(device)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoded, encoder_hidden = encoder(slang, encoder_hidden)
    decoded, decoder_hidden = decoder(target, encoder_hidden)
    
    
    target_len = target.shape[0]
    for i in range(target_len):
      
      loss += criterion(decoded[0][i].view(-1), target[i].view(-1))
      loss.backward()
      encoder_optimizer.step()
      decoder_optimizer.step()
      running_loss += loss.item()

    
    
  else:
    test_loss = 0 
    with torch.no_grad():
      for slang, target in zip(to_torch(X_test), to_torch(y_test)):
        slang = slang.to(device)
        target = target.to(device)
        encoded, encoder_hidden = encoder(slang, encoder_hidden)
        decoded, decoder_hidden = decoder(target, encoder_hidden)
        target_len = target.shape[0]
        for i in range(target_len):
          loss = criterion(decoded[0][i].view(-1), target[i].view(-1))
          test_loss += loss.item()
        
        res = decoded.view(*target.shape)
        equals = res == target
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
    print('epoch: {} ---------------'.format(e))
    train_losses.append(running_loss/X_train.shape[0])
    test_losses.append(test_loss/X_test.shape[0])
    
    
  
        
  
  
  

In [0]:
tembed.embedding(to_torch(X_train[:1])[0]).view(1,-1).shape

In [0]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [0]:
e = nn.Embedding(len(vocab), 10)
l = nn.Linear(20, 128)
context, target = trigrams[0]
context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
print(context_idxs.shape)
_e = e(context_idxs).view((1, -1))
print(_e.shape)
_e = l(_e)
print(_e.shape)
_e


In [0]:
len(vocab)

In [0]:
trigrams[0]

In [0]:
len(X_train[0])

In [0]:
np.max(
    np.unique(np.array(X.values.tolist()).ravel())
)

122